# Plant Pathology Hypertuning

### Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
import copy

from utils import utils, learner
from utils import PlantPathologyDataset as dataset

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

/home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
print_every = 100
assert(torch.cuda.is_available())
gpu_dtype = torch.cuda.FloatTensor

In [3]:
NUM_TRAIN = 75
NUM_VAL = 25
users = ('braedon', 'thomas', 'shangyi')
user = users[0]

if user == users[1]:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
elif user == users[0]:
    csv_file='/home/braedon/skole/cs175/PlantPathology2021/train.csv'
    root_dir='/home/braedon/skole/cs175/PlantPathology2021/train_images'
    mappings_dir='/home/braedon/skole/cs175/PlantPathology2021/labelMappings.csv'

plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir)
train_data = DataLoader(plant_dataset, batch_size=10, sampler= learner.ChunkSampler(NUM_TRAIN, 0))
validation_data = DataLoader(plant_dataset, batch_size=10, sampler=learner.ChunkSampler(NUM_VAL, NUM_TRAIN))

### Testing resnet50

In [6]:
resnet50 = models.resnet50()#.cuda() # pretrained=True is pretrained on ImageNet

loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)

In [7]:
learner.train(resnet50, train_data, loss_fn, optimizer, print_every=5)
learner.check_accuracy(resnet50, validation_data)

Starting epoch 1 / 1


KeyboardInterrupt: 

### Hypertuning

In [8]:
# This function is called by raytune in the hyperparameter tuning. 
def train_resnet(config):
    model = models.resnet50()
    # train_loader, test_loader = get_data_loaders()
    loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"])#, momentum=config["momentum"])
    # optimizer = optim.RMSprop(model.parameters(), lr=1e-3)
    for i in range(10):
        learner.train(model, train_data, loss_fn, optimizer, print_every=5)
        acc = learner.check_accuracy(model, validation_data)
        tune.report(mean_accuracy=acc)
        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")

In [9]:
# Add hyperparameters here for additional tuning
# change their values to adjust the range and selection mechanism (uniform between two points, uniform from given options, iterate through all given options, etc.)
search_space = {
    "lr": tune.choice([0.1])
}
num_samples = 1

# This is what does the hyperparameter tuning. Right now, it's set to call the train_resnet function, and it works toward maximizing the mean_accuracy. It can also minimize loss.
analysis = tune.run(
    train_resnet,
    num_samples=num_samples,
    scheduler=ASHAScheduler(metric="mean_accuracy", mode="max", grace_period=1),
    config=search_space)

2021-05-14 13:09:55,443	WARNING tune.py:494 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,lr
train_resnet_59a9c_00000,RUNNING,,0.1


(pid=3096) /home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
(pid=3096)   from .collection import imread_collection_wrapper
(pid=3096) Starting epoch 1 / 1
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
(pid=3096) t = 5, loss = 12.7788
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
(pid=3096) torch.Size([10, 3, 400, 267]) ten

Trial name,status,loc,lr,acc,iter,total time (s)
train_resnet_59a9c_00000,RUNNING,192.168.1.10:3096,0.1,0.28,1,305.618


(pid=3096) Got 7 / 25 correct (28.00)
(pid=3096) Starting epoch 1 / 1
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
(pid=3096) t = 5, loss = 12.6559
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
(pid=3096) torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
(pid=3096) Checking accuracy on validation set
Result for train_resnet_59a9c_00000:
  date: 2021-05-14_13-20-07
  done: false
  experiment_id: 00be260e02a84a7cb8d34a9f40613567
  hostname: jeremiah-v2-linux
  iterations_since_restore: 2
  mean_

...

(pid=3096) Got 8 / 25 correct (32.00)
(pid=3096) Starting epoch 1 / 1
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
(pid=3096) t = 5, loss = 2.3092
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
(pid=3096) torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
(pid=3096) torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
(pid=3096) Checking accuracy on validation set
Result for train_resnet_59a9c_00000:
  date: 2021-05-14_13-59-52
  done: false
  experiment_id: 00be260e02a84a7cb8d34a9f40613567
  hostname: jeremiah-v2-linux
  iterations_since_restore: 10
  mean_

Trial name,status,loc,lr,acc,iter,total time (s)
train_resnet_59a9c_00000,RUNNING,192.168.1.10:3096,0.1,0.32,10,2995.42


Result for train_resnet_59a9c_00000:
  date: 2021-05-14_13-59-52
  done: true
  experiment_id: 00be260e02a84a7cb8d34a9f40613567
  experiment_tag: 0_lr=0.1
  hostname: jeremiah-v2-linux
  iterations_since_restore: 10
  mean_accuracy: 0.32
  node_ip: 192.168.1.10
  pid: 3096
  time_since_restore: 2995.4186906814575
  time_this_iter_s: 296.73072934150696
  time_total_s: 2995.4186906814575
  timestamp: 1621025992
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 59a9c_00000
  


Trial name,status,loc,lr,acc,iter,total time (s)
train_resnet_59a9c_00000,TERMINATED,,0.1,0.32,10,2995.42


2021-05-14 13:59:52,139	INFO tune.py:549 -- Total run time: 2996.72 seconds (2996.59 seconds for the tuning loop).
(pid=3096) Got 8 / 25 correct (32.00)


In [ ]:
# To do (analysis)...

# print("Best config: ", analysis.get_best_config(
#     metric="mean_loss", mode="min"))

# # Get a dataframe for analyzing trial results.
# df = analysis.results_df